In [2]:
import simpy
import itertools
import numpy as np

## SINGLE-SERVER QUEUEING SYSTEM

Processes
- arrival 
- service

#### Generating all cars

In [3]:
def car(env, name, bcs, driving_time, charge_duration):
    # Simulate driving to Charging Station
    yield env.timeout(driving_time)
    
    # Request one of it's charging stations
    print('car {} is arriving at {:.0f}'.format(name, env.now))
    with bcs.request() as req:
        yield req
        
        # Charge the battery
        print('starting to charge Car {} at {}'.format(name, env.now))
        yield env.timeout(charge_duration)
        print('Car {} finished charging at {}'.format(name, env.now))
    

In [4]:
env = simpy.Environment()
bcs = simpy.Resource(env, capacity=2)
env.process(car(env, "1", bcs, 1, 5))
env.process(car(env, "2", bcs, 2, 5))
env.process(car(env, "3", bcs, 3, 5))
env.process(car(env, "4", bcs, 4, 5))
env.run(until=60)

car 1 is arriving at 1
starting to charge Car 1 at 1
car 2 is arriving at 2
starting to charge Car 2 at 2
car 3 is arriving at 3
car 4 is arriving at 4
Car 1 finished charging at 6
starting to charge Car 3 at 6
Car 2 finished charging at 7
starting to charge Car 4 at 7
Car 3 finished charging at 11
Car 4 finished charging at 12


#### with car generator

In [66]:
arrival_times = []
waiting_times = []
service_times = []

In [67]:
def car(env, name, queue, charge_duration):
    arrival_time = env.now
    arrival_times.append(arrival_time)
    print('{} is arriving at {:.2f}'.format(name, env.now))
    with queue.request() as req:
        yield req
        
    # Start serving
    waiting_time = env.now - arrival_time
    print('starting to serve {} at {:.2f}'.format(name, env.now))
    waiting_times.append(waiting_time)
        
    # Finish serving
    yield env.timeout(np.random.exponential(charge_duration))
    print('{} finished serving at {:.2f}'.format(name, env.now))
        
    service_times.append(env.now - waiting_time - arrival_time)
    

In [68]:
def car_generator(env, queue):
    interarrival_time = 10
    charge_duration = 5
    """Generate new cars that arrive at the gas station."""
    for i in itertools.count():
#     for i in range(1,5):
        yield env.timeout(np.random.exponential(10))
        
        env.process(car(env, 'Car %d' % i, queue, charge_duration ))
    return i

In [69]:
env = simpy.Environment()
queue = simpy.Resource(env, capacity=1)
proc = env.process(car_generator(env, queue))
env.run(until=60)

Car 0 is arriving at 6.64
starting to serve Car 0 at 6.64
Car 1 is arriving at 13.04
starting to serve Car 1 at 13.04
Car 1 finished serving at 18.20
Car 0 finished serving at 18.85
Car 2 is arriving at 33.56
starting to serve Car 2 at 33.56
Car 2 finished serving at 34.60
Car 3 is arriving at 53.14
starting to serve Car 3 at 53.14
Car 3 finished serving at 56.63


In [70]:
arrival_times

[6.64477617237064, 13.039445867562257, 33.56497880826073, 53.14123228807745]

In [71]:
waiting_times

[0.0, 0.0, 0.0, 0.0]

In [72]:
service_times

[5.158162338179579, 12.203846815848586, 1.0378199138162145, 3.492224707584967]

## SIMULATION OF AN INVENTORY SYSTEM